In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from pathlib import Path

from astropy import units as u
from astropy.table import Table 
from astropy.nddata import CCDData
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.time import Time, TimeDeltaSec

import glowing_waffles.differential_photometry as gwdf


## Change the settings below as appropriate for your images

In [ ]:
# This can be a relative or absolute path
# On Windows make this a "raw" string by putting an r before the string, otherwise
# Python interprets the backslashes as escapotential energy sequences.
# Windows example:
# directory_with_images = r'C:\path\to\my\files'
#
# Mac/Linux example:
directory_with_images = 'test_sub'

# If your magnitudes are in the same directory as the images then do
# directory_with_magnitude_file = directory_with_images
directory_with_magnitude_file = '.'

# This only used to center to make a light curve of your object at the end of this
# notebook.
# If the object you have can't be looked up in simbad this cell will raise an error.
#
# If it does raise an error, then set the object of interest to an empty string
object_of_interest = 'kelt-16b'

if object_of_interest:
    obj_coords = SkyCoord.from_name(object_of_interest)
    
sample_image_for_finding_stars = 'kelt-16-S001-R001-C100-r.fit'

input_photometry_file = 'all_the_photometry_kelt-16.fits'

# Set the faintest stars to use in calibrating the magnitudes
# For the new camera (any nights in 2018) use 13
# For the old camera (any older nights) use 16
faintest_mag = 13

# Set the name of the apass column to use for calibration
# Should be r_mag for the new camera, R_mag for the old
apass_cal_mag = 'r_mag'

# Set the name of thecolumn in our data file for our instrumental magnitude
# Should be mag_inst_r for the new camera, mag_inst_R for the old
feder_instrumental_mag = 'mag_inst_r'

# This must be a different file name
output_photometry_file = 'some_good_name.fits'

In [ ]:
ccd = CCDData.read(Path(directory_with_images) / sample_image_for_finding_stars)

In [ ]:
apass, apass_x, apass_y, apass_for_transform, in_apass_x, in_apass_y = gwdf.find_apass_stars(ccd)
vsx, vsx_x, vsx_y, vsx_names = gwdf.find_known_variables(ccd)

apass_all_coords = SkyCoord(apass['RAJ2000'], apass['DEJ2000'], unit='deg')
apass_for_transform_coords = SkyCoord(apass_for_transform['RAJ2000'], apass_for_transform['DEJ2000'], unit='deg')

## Add column for R filter in case we need it

In [ ]:
apass['R_mag'] = gwdf.filter_transform(apass, 'R', r='r_mag', i='i_mag', transform='ivezic')
apass_for_transform['R_mag'] = gwdf.filter_transform(apass_for_transform, 'R', r='r_mag', i='i_mag', transform='ivezic')

## Read in the data file

In [ ]:
input_mags = Table.read(Path(directory_with_magnitude_file) / input_photometry_file)

## Get our "star_id" for object of interest

In [ ]:
if object_of_interest:
    input_coords = SkyCoord(input_mags['RA'], input_mags['Dec'])
    idx, d2d, d3d = object_of_interest_coords.match_to_catalog_sky(input_coords)

    object_of_interest_id = input_mags['star_id'][idx]
    print(object_of_interest_id)

In [ ]:
input_mags['mag_cal'] = np.zeros(len(input_mags))
input_mags['zero_point'] = np.zeros(len(input_mags))
input_mags['color_term'] = np.zeros(len(input_mags))
input_mags['t_start_tdb'] = np.zeros(len(input_mags))
input_mags['t_mid_tdb'] = np.zeros(len(input_mags))

mags_grouped = input_mags.group_by('file')

In [ ]:
feder = EarthLocation("263.54672d", "46.86678d")

In [ ]:
input_mags.colnames

In [ ]:
print('On image...', end='')
for idx, our_mags in enumerate(mags_grouped.groups):
    print(str(idx) + '..', end='')
    new_mags, matches, transform = gwdf.transform_magnitudes(our_mags, apass, apass_for_transform, 
                                                faintest_mag_for_transform=faintest_mag,
                                                input_mag_colum=feder_instrumental_mag,
                                                catalog_mag_column=apass_cal_mag,
                                                catalog_color_column='B-V'
                                               )
    our_mags['mag_cal'][matches] = new_mags.data.filled(np.nan)
    our_mags['mag_cal'][~matches] = np.nan
    our_mags['zero_point'] = transform.parameters[0]
    our_mags['color_term'] = transform.parameters[1]
    t_utc = Time(our_mags['date-obs'], scale='utc', location=feder)
    our_coords = SkyCoord(our_mags['RA'], our_mags['Dec'])
    delta_t_tdb = t_utc.light_travel_time(our_coords)
    t_tdb = t_utc.tdb + delta_t_tdb
    t_tdb_mid = t_tdb + our_mags['exposure'] / 2
    our_mags['t_start_tdb'][:] = t_tdb.jd
    our_mags['t_mid_tdb'][:] = t_tdb_mid.jd

In [ ]:
mags_grouped.sort('t_start_tdb')

In [ ]:
if object_of_interest:
    object_alone = mags_grouped[mags_grouped['star_id'] == object_of_interest_id]
    

## Identify known variables in the field 

### Do the matching...

In [ ]:
mag_coords = SkyCoord(mags_grouped['RA'], mags_grouped['Dec'])
vsx_coords = SkyCoord(vsx['RAJ2000'], vsx['DEJ2000'])
vsx_idx, d2d, d3d = mag_coords.match_to_catalog_sky(vsx_coords)
matches = d2d < 3 * u.arcsec

### Add VSX name column to output table

In [ ]:
vsx_names = np.array([''] * len(mags_grouped), dtype='S50')
vsx_names[matches] = vsx['Name'][vsx_idx[matches]]
mags_grouped['vsx_name'] = vsx_names

## Write output to a file

In [ ]:
mags_grouped.write(output_photometry_file)

## Graph transform coefficients


In [ ]:
regrouped = mags_grouped.group_by('file')
agg_regrouped = regrouped.groups.aggregate(np.mean)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(agg_regrouped['color_term'], '.')
#plt.ylim(-0.05, 0.05)
plt.grid()

In [ ]:
plt.figure(figsize=(15, 5))
plt.grid()
plt.plot(agg_regrouped['t_mid_tdb'], agg_regrouped['zero_point'], '.')
#plt.ylim(20, 21)

## Graph of your object, *uncalibrated* magnitudes

In [ ]:
if not object_of_interest:
    raise ValueError('Remaining cells will work only if the object_of_interest is set to a non-empty string.')

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(object_alone['t_mid_tdb'], object_alone['mag_inst_r'], '.')
plt.ylim(reversed(plt.ylim()))
plt.title('REPLACE THIS TEXT WITH MEANINGFUL TITLE')
plt.ylabel('instrumental magnitude, r filter')
plt.xlabel('barycentric julian date')
plt.grid()

## Graph of your object, *calibrated* magnitudes

In [ ]:
plt.figure(figsize=(15, 5))

plt.plot(object_alone['t_mid_tdb'], object_alone['mag_cal'], '.')
plt.ylim(reversed(plt.ylim()))
plt.title('REPLACE THIS TEXT WITH MEANINGFUL TITLE')
plt.ylabel('calibrated magnitude, r filter')
plt.xlabel('barycentric julian date')
plt.grid()